# Xử lý dữ liệu

B1: Làm sạch dữ liệu theo thứ tự:
+ Loại bỏ emoji.
+ Lower các kí tự.
+ Loại bỏ các tiếng viết lặp nguyên âm hay phụ âm như ngooon hay biếttt.
+ Thay biểu thức có dạng giá tiền bằng từ giá thành. Ví dụ: 5k/cốc => giá thành cốc.
+ Loại bỏ số.
+ Loại bỏ url.
+ Thay thế một số từ viết tắt và sai chính tả thông dụng từ replace_dict.
+ Loại bỏ dấu câu.
+ Loại bỏ từ viết tắt.
+ Tokenizer

B2:
    Lưu chúng lại trong một fike csv có tên clean_comment.csv.
    
B3: 
    Tạo một word2vec embedding


In [ ]:
import re
import string
import codecs
import emoji
import unicodedata
import gensim
import operator
import pandas as pd
import numpy as np
from underthesea import word_tokenize, pos_tag, sent_tokenize

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
df = pd.read_csv('../data_clean/comment_with_score.csv')

**Loại bỏ emoji:**

In [ ]:
def replace_icon(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
        
    return text
df['review'] = df['review'].apply(replace_icon)

**Lower các kí tự**:

In [ ]:
df['review'] = df['review'].str.lower()

**Thay thế tiền bằng từ giá thành:**

In [ ]:
df['review'] = df['review'].replace(r'[0-9]k', ' giá thành ', regex=True)

**Loại bỏ số:**

In [ ]:
df['review'] = df['review'].replace('\d', '', regex=True)

**Loại bỏ url:**

In [ ]:
df['review'] = df['review'].replace('http://\S+|https://\S+', '', regex=True)

**Sửa các từ có kí tự trùng lặp:**

In [ ]:
df["review"] = df["review"].str.replace(r'(\w)\1+', lambda m: m.group(1), regex=True)

**Thay thế viết tắt và sai chính tả:**

In [ ]:
def replace_text(text):  
    replace_dict = np.load('../dicts/replace_dict.npy',
                           allow_pickle='TRUE').item()
    
    for k, v in replace_dict.items():
        text = text.replace(k, v)

    return text

In [ ]:
df['review'] = df['review'].apply(replace_text)

**Loại bỏ dấu câu:**

In [ ]:
df['review'] = df['review'].str.replace('[{}]'.format(string.punctuation), ' ')

**Tokenizer:**

In [ ]:
df['review'] = df['review'] = df['review'].apply(lambda x: word_tokenize(x, format='text'))

**Loại bỏ stopwords:**

In [ ]:
stop_word = ['các', 'những', 'rằng', 'và', 'thì', 'là', 'mà', 'thì',
             'cô', 'dì', 'chú', 'bác', 'anh', 'chị', 'em', 'mình', 'tớ', 'cậu',
             'đây', 'đó', 'kia', 'cũng', 'vậy', 'nên', 
             'and', 'the', 'is', 'it', 'les', 'tous', 'for', 'ours', 'of', 'you',
             'huhu', 'hjhj', 'haha', 'he he', 'hehe' ,'hihi', 'với', 'vài', 'chiếc',
             'cái', 'bởi', 'vì', 'nên', 'lắm', 'hichic', 'hic hic', 'hehehe'
             ]

In [ ]:
df['review'] = df['review'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_word]))

In [ ]:
df.dropna(inplace=True)
df = df[df['review']!='']

**Ta được:**

In [ ]:
df.head()

**Lưu lại vào file csv:**

In [ ]:
df.to_csv('clean_review.csv', index=False)

In [ ]:
input_gensim = []
for review in df['review']:
    list_word = []
    for word in review.split():
        list_word.append(word.strip())
    input_gensim.append(list_word)

In [ ]:
model = gensim.models.Word2Vec(input_gensim, size=200, window=5, min_count=0, workers=4, sg=1)
vectors = model.wv

In [ ]:
model_file = '../w2v_embedding' 
model.save(model_file) 